In [19]:
import random
import string 
import collections
from bisect import bisect_left
import time
import multiprocessing

#function to get the dictionary words from file "words.txt" letter wise
def Dictionary(fname="words.txt"):
    words = collections.defaultdict(list)
    with open(fname) as f:
        for word in f.readlines():
            if len(word) > 3:
                words[word[0]].append(word.strip())
    return words

#generating a random grid of desired size
def Grid(size=8):
    grid = [[random.choice(string.ascii_lowercase) for c in range(size)] for r in range(size)]
    return grid

#Given: grid, position of letter whose adjecents are to be found(r,c)
#returns turple of (letter, row index, column index) for each index of given letter
#origin: top left corner
def AdjacentLetters(grid, r, c):
    gsize = len(grid)
    # indexes of all neighbours for a given cell
    neighbours = [ [(r-1,c-1),(r-1,c),(r-1,c+1)],
                   [(r,c-1),          (r,c+1)],
                    [(r+1,c-1),(r+1,c) ,(r+1,c+1)] ]

    #removing non existing neighbours
    if c == 0:
        res = [n[1:] for n in neighbours]
    elif c == gsize-1:
        res = [neighbours[0][0:2], [neighbours[1][0]], neighbours[2][0:2]]
    else:
        res = neighbours
    if r == 0:
        res = res[1:]
    elif r == gsize-1:
        res = res[0:2]
    else:
        pass

    letters = []
    for row in res:
        letters.extend( [(grid[r][c],r,c) for r,c in row] )
    return letters

def PrefixMatches(word, dictionary):
    return [m for m in dictionary[word[0]] if m.startswith(word)]

def isValidWord(word,dictionary):
    return word in dictionary[word[0]]

def getWords(grid,dictionary,seed_letter,ply=64):
    # recursive function using depth first traversal
    def genCandidates(results, seed_letter, ignored, curWord=""):
        if len(curWord) > ply:
            " -> maximum ply of %s reached, baling out" % ply
            return
        else:
            print("  trying ",curWord)
        if isValidWord(curWord,dictionary):
            results.add(curWord)
            print("    -> match: ",curWord)
            
        # make sure we dont visit the same letter twice
        ignored.add(seed_letter)

        # the neighbours of the seed cell
        neighbours = getAdjacentLetters(grid,seed_letter[1],seed_letter[2])
        
        # remove cells we have visited before by checking against the ignored list
        candidates = [n for n in neighbours if n not in ignored]
        
        # see how many words we can find for each candidate prefix
        num_matches = [len(getPrefixMatches(curWord + n[0], dictionary)) for n in candidates]
        
        # zip them together in a convenient list of tuples
        matches = zip(candidates,num_matches)
        
        # for each candidate with more than 0 matches recursively call the function
        for c,numMatches in matches:
            if numMatches > 0:
                genCandidates(results,c,ignored,curWord=curWord + c[0])

    results = set()
    genCandidates(results,seed_letter,set(),curWord=seed_letter[0])
    print("**********")
    print("Seed: ", seed_letter)
    print("--> %s words" % len(results))
    print("**********")
    return results

def solve(grid,dictionary,seeds,ply):
    totResults = set()
    
    # iterate over every seed sequentially
    for seed in seeds:
        # the actual computation			
        results = getWords(grid,dictionary,seed,ply=ply)
        totResults = totResults.union(results)
    return totResults

#to display grid
def print_grid(grid):
    for r in grid:
        print(r)



dictionary = getDictionary()
gsize = 8  # grid size
ply = 64   # max word length

# a random grid
grid = Grid(size=gsize)

# display the grid
print_grid(grid)

# build the list of seed tuples (letter,row_idx,col_idx) for every item in the grid
seeds = []
for r,row in enumerate(grid):
    for c,letter in enumerate(row):
        seeds.append( (letter,r,c) )

# start the timer
tstart = time.time();

# Solving boggle
results = solve(grid,dictionary,seeds,ply)

print("=======================================================================================================")
print("%s words with max %s letters found in a %sx%s grid after %s seconds: " % (len(results),ply,gsize,gsize,time.time() - tstart))
print()
print("   ",",".join(sorted(results)))
print("=======================================================================================================")

['q', 'l', 'f', 'o', 'p', 'c', 'q', 'h']
['b', 's', 't', 'o', 'n', 'c', 'z', 'y']
['g', 'h', 'r', 'g', 'e', 'i', 'c', 'b']
['z', 'i', 'm', 'u', 'l', 'v', 'r', 'b']
['b', 'z', 'q', 's', 'z', 'k', 'q', 'r']
['b', 'r', 'k', 'v', 'n', 'a', 'l', 'z']
['v', 't', 'c', 'u', 'a', 'f', 'f', 'y']
['x', 'a', 'c', 'm', 'h', 'x', 'c', 't']
  trying  q
**********
Seed:  ('q', 0, 0)
--> 0 words
**********
  trying  l
  trying  lb
  trying  lbs
    -> match:  lbs
**********
Seed:  ('l', 0, 1)
--> 1 words
**********
  trying  f
  trying  fl
  trying  fo
  trying  fop
    -> match:  fop
  trying  foo
  trying  foot
    -> match:  foot
  trying  foots
    -> match:  foots
  trying  footh
  trying  foothi
  trying  footr
  trying  footg
  trying  footge
  trying  fon
  trying  ft
  trying  fo
**********
Seed:  ('f', 0, 2)
--> 3 words
**********
  trying  o
  trying  of
  trying  oft
    -> match:  oft
  trying  op
  trying  opc
  trying  opo
  trying  opc
  trying  ot
  trying  oth
  trying  oo
  trying  o

  trying  len
  trying  leng
  trying  lec
  trying  leg
    -> match:  leg
  trying  legr
  trying  legm
  trying  legu
  trying  lei
    -> match:  lei
  trying  leu
  trying  lev
  trying  li
  trying  liz
  trying  lic
  trying  lir
  trying  lu
  trying  lus
  trying  lv
**********
Seed:  ('l', 3, 4)
--> 2 words
**********
  trying  v
  trying  ve
  trying  ven
  trying  veno
  trying  veno
  trying  veng
  trying  veni
  trying  venir
  trying  vec
  trying  veg
  trying  vei
  trying  veil
    -> match:  veil
  trying  veils
    -> match:  veils
  trying  vel
  trying  velu
  trying  velum
    -> match:  velum
  trying  vi
  trying  viz
  trying  vic
  trying  vr
**********
Seed:  ('v', 3, 5)
--> 3 words
**********
  trying  r
  trying  ri
  trying  rin
  trying  ring
    -> match:  ring
  trying  ringt
  trying  ringe
  trying  ringm
  trying  ringl
  trying  ric
  trying  ric
  trying  ril
  trying  riv
**********
Seed:  ('r', 3, 6)
--> 1 words
**********
  trying  b
  trying 